In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#Clear all variables from memory, Do not run unless you want to remove all memory.
from IPython import get_ipython;   
get_ipython().magic('reset -sf')

In [2]:

import pandas as pd
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 140)

In [3]:
debug = 'true'
%whos

Variable   Type        Data/Info
--------------------------------
HTML       type        <class 'IPython.core.display.HTML'>
debug      str         true
display    function    <function display at 0x000001BD935CFAE8>
pd         module      <module 'pandas' from 'c:<...>es\\pandas\\__init__.py'>


In [4]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd

#import sys
#import matplotlib as plt

root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 300, bg = 'lightsteelblue')
canvas1.pack()

def getExcel ():
    global df
    
    import_file_path = filedialog.askopenfilename()
    df = pd.read_excel (import_file_path)
    #Convert headings from Number values into String Values. Makes it simpler to reference a String
    df.columns = df.columns.astype(str)
    print (df.head(0))
    
browseButton_Excel = tk.Button(text='Import Excel File', command=getExcel, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 150, window=browseButton_Excel)

root.mainloop()

del root
del canvas1              
del browseButton_Excel
del filedialog   
del getExcel     

del tk         

Empty DataFrame
Columns: [PSLK, 201207, 201208, 201209, 201210, 201211, 201212, 201301, 201302, 201303, 201304, 201305, 201306, 201307, 201308, 201309, 201310, 201311, 201312, 201401, 201402, 201403, 201404, 201405, 201406]
Index: []

[0 rows x 25 columns]


In [5]:
startDate = 2012
YearsToObserve = 8

admissionStartYear = []
aStartYear = []
aEndYear = []
DischargeEndYear = []

for i in range(YearsToObserve):
    
    admissionStartYear.append(str(startDate) + '06')
    
    aStartYear.append( str(startDate) +'07')
    aEndYear.append( str(startDate + 1) +'06')
    
    DischargeEndYear.append( str(startDate + 1) +'07')
    
    startDate += 1
       
#initialise data of lists. 
data = {'AdmissionStartYear': admissionStartYear, 'StartYear':aStartYear, 
        'EndYear':aEndYear, 'DischargeEndYear':DischargeEndYear   } 

#Creates pandas DataFrame. 
dfYearsToInspect = pd.DataFrame(data) 
  
if debug=='true':
    print(dfYearsToInspect)
    
    print('Rows ' , dfYearsToInspect.shape[0]) #The number of Rows
    #print('Columns', dfYearsToInspect.shape[1]) #The Number of Columns

  AdmissionStartYear StartYear EndYear DischargeEndYear
0             201206    201207  201306           201307
1             201306    201307  201406           201407
2             201406    201407  201506           201507
3             201506    201507  201606           201607
4             201606    201607  201706           201707
5             201706    201707  201806           201807
6             201806    201807  201906           201907
7             201906    201907  202006           202007
Rows  8
Columns 4


In [6]:
def DefineDataFrameColumn(dataFrame, columnName):
    PSLK = pd.DataFrame(dataFrame, columns= [columnName], dtype=int)
    PSLK.describe()

In [7]:
def Admissions(row, colA, colB):
    if row[colA] < row[colB]:
        val = 1
    else:
        val = 0
    return val

In [8]:
def Discharges(row, colA, colB):
    if row[colA] > row[colB]:
        val = 1
    else:
        val = 0
    return val

In [9]:
def IdentifyColumns(dataFrame,  colToIgnore):
    aCol = []
    for (columnName, columnData) in df.iteritems():
        if columnName != colToIgnore: 
            aCol.append(columnName)

    return aCol

In [10]:
# Returns a new list. "lst" is not modified.
def delete__by_values(list, valuesToDelete):
    values_as_set = set(valuesToDelete)
    return [ x for x in list if x not in values_as_set ]

In [11]:
# Returns a new list. "lst" is not modified.
def Retain__by_values(list, valuesToRetain):
    values_as_set = set(valuesToRetain)
    return [ x for x in list if x in values_as_set ]

In [12]:
aHeadings = IdentifyColumns(df.copy(),  'PSLK')

if debug=='true':
    print(aHeadings)

['201207', '201208', '201209', '201210', '201211', '201212', '201301', '201302', '201303', '201304', '201305', '201306', '201307', '201308', '201309', '201310', '201311', '201312', '201401', '201402', '201403', '201404', '201405', '201406']


In [13]:
#perform sum of all rows for *Admissions and *Discharges
dfTest = df.copy()

col_list = list(df)

col_list = delete__by_values( aHeadings, 'PSLK' )

listOfAdmissionColumns = []
listOfDischargeColumns = []
#total =  dfTestAdmissions[col_list].sum(axis=1)

for i in range(len(col_list)-1):
    print('Processing Years: ', col_list[i], col_list[i+1]) 
    
    #This just means, select the first four characters of the String Variable
    AdmissionHeading = 'A' +  str(col_list[i][-4:]) + '-' + str(col_list[i+1][-4:]) 
    DischargeHeading = 'D' +  str(col_list[i][-4:]) + '-' + str(col_list[i+1][-4:]) 
  
    dfTest[AdmissionHeading] = dfTest.apply(Admissions,colA=col_list[i],colB=col_list[i+1],axis=1)
    dfTest[DischargeHeading] = dfTest.apply(Discharges,colA=col_list[i],colB=col_list[i+1],axis=1)

    #This just means, select the first four characters of the String Variable
    
    if AdmissionHeading not in listOfAdmissionColumns:
         listOfAdmissionColumns.append(AdmissionHeading)    
    
    if DischargeHeading not in listOfDischargeColumns:
         listOfDischargeColumns.append(DischargeHeading)

    
with pd.option_context('display.max_rows', 4, 'display.max_columns', None):  # more options can be specified also
    print(dfTest)


Processing Years:  201207 201208
Processing Years:  201208 201209
Processing Years:  201209 201210
Processing Years:  201210 201211
Processing Years:  201211 201212
Processing Years:  201212 201301
Processing Years:  201301 201302
Processing Years:  201302 201303
Processing Years:  201303 201304
Processing Years:  201304 201305
Processing Years:  201305 201306
Processing Years:  201306 201307
Processing Years:  201307 201308
Processing Years:  201308 201309
Processing Years:  201309 201310
Processing Years:  201310 201311
Processing Years:  201311 201312
Processing Years:  201312 201401
Processing Years:  201401 201402
Processing Years:  201402 201403
Processing Years:  201403 201404
Processing Years:  201404 201405
Processing Years:  201405 201406
      PSLK  201207  201208  201209  201210  201211  201212  201301  201302  201303  201304  201305  201306  201307  201308  201309  \
0        1       0       1       1       1       1       1       1       1       1       1       1       0 

In [15]:
#This formula is corret, we just need to perform the wider calculation on the tables with this information.

#print(dfYearsToInspect)
# Admissions Looks behind : Start on YYYY07; giving preference to YYYY06 over YYYY07 End on (YYYY+1)06

aAdmission = pd.DataFrame(columns=('Index', 'StartYear', 'EndYear', 'Type'))

for index,row in dfYearsToInspect.iterrows():

    #give preference to the AdmissionsStartYear. Only take a StartYear if ASY does not exist
    if row['AdmissionStartYear'] in aHeadings and row['EndYear'] in aHeadings:
        aAdmission.loc[index] = index, row['AdmissionStartYear'], row['EndYear'] , 'Admission'
            
    elif row['StartYear'] in aHeadings and row['EndYear'] in aHeadings:
        aAdmission.loc[index] =index,  row['StartYear'], row['EndYear']  , 'Admission'
        
if debug=='true':
    print (aAdmission)


  Index StartYear EndYear       Type
0     0    201207  201306  Admission
1     1    201306  201406  Admission


In [14]:
# Discharges look ahead. Start with YYYY07 -> through to (YYYY+1)06, giving preference to (YYYY+1)07 over (YYYY+1)06

aDischarge = pd.DataFrame(columns=('Index', 'StartYear', 'EndYear', 'Type'))

for index,row in dfYearsToInspect.iterrows():

    if row['StartYear'] in aHeadings and row['DischargeEndYear'] in aHeadings:
        aDischarge.loc[index] = index, row['StartYear'], row['DischargeEndYear']  , 'Discharge'   
  
    elif row['StartYear'] in aHeadings and row['EndYear'] in aHeadings:
        aDischarge.loc[index] = index, row['StartYear'], row['EndYear'], 'Discharge'

if debug=='true':
    print (aDischarge)
    
#if index == 1 : break #for testing, just use the first returned value.

  Index StartYear EndYear       Type
0     0    201207  201307  Discharge
1     1    201307  201406  Discharge


In [ ]:
# Admissions Looks behind : Start on YYYY07; giving preference to YYYY06 over YYYY07 End on (YYYY+1)06
debug='false'
# print(aHeadings)

AdmissionIndexes = pd.DataFrame(columns=('Index', 'YearListing', 'Type'))

for index,row in aAdmission.iterrows():
   # print(index,row.StartYear, row.EndYear, row.Count)
    if row['StartYear'] in aHeadings:

        startingPosition = aHeadings.index(row['StartYear'])
        endingPosition = aHeadings.index(row['EndYear'])
        AdmissionIndexes.loc[index] = index, aHeadings[startingPosition:endingPosition+1], 'Admission'

        if debug=='true':
            print(aHeadings.index(row['StartYear']) ,  aHeadings.index(row['EndYear']) )
            print(aHeadings[startingPosition:endingPosition+1]   )
            #AdmissionIndexes.append(aHeadings[startingPosition:endingPosition+1])
            
      # if index == 0 : break #for testing, just use the first returned value.
   
    # for index, row in enumerate(headings):
#     if row in aHeadings:
#         indexes.append(index)
if debug=='true':
    print(AdmissionIndexes)

In [ ]:
# Discharges look ahead. Start with YYYY07 -> through to (YYYY+1)06, giving preference to (YYYY+1)07 over (YYYY+1)06

#print(aHeadings)

DischargeIndexes = pd.DataFrame(columns=('Index', 'YearListing', 'Type'))

for index,row in aDischarge.iterrows():
    # print(index,row.StartYear, row.EndYear, row.Count)
    if row['StartYear'] in aHeadings:

        startingPosition = aHeadings.index(row['StartYear'])
        endingPosition = aHeadings.index(row['EndYear'])
        DischargeIndexes.loc[index] = index, aHeadings[startingPosition:endingPosition+1], 'Discharge'
            
        if debug=='true':
            print(aHeadings.index(row['StartYear']) ,  aHeadings.index(row['EndYear']) )
            print(aHeadings[startingPosition:endingPosition + 1]   )
           
      # if index == 0 : break #for testing, just use the first returned value.
    
# if debug=='true':
print(DischargeIndexes)   

In [ ]:
#loop all AdminissionIndexes to calculate the required Numbers
debug = 'false'


#reset the testing Data Frame.
dfTestAdmissions = df.copy()

#numCols = AdmissionIndexes.shape[0]

#This is the first Year listing. ill need to loop over all Year Listing!

#print(AdmissionIndexes)
for i in range(len(AdmissionIndexes.YearListing)  ): 
    if debug=='true':
        print(i, AdmissionIndexes.YearListing[i])
        print('Start of New Section')

    AdmissionYears = AdmissionIndexes.YearListing[i]
    
    listOfColumns = []
    
    for j in range(len(AdmissionYears) -1 ): 
        
        dColA = str(AdmissionYears[j])
        dColB = str(AdmissionYears[j + 1])
        
        if debug=='true':
            print('Identified Year Pairs:', AdmissionYears[j], AdmissionYears[j+1])       
            print('d', dColA, dColB)

        #This just means, select the first four characters of the String Variable
        AdmissionHeading = 'A' +  str(dColA[-4:]) + '-' + str(dColB[-4:]) 
        
        if debug=='true':
            print(AdmissionHeading)
        
        dfTestAdmissions[AdmissionHeading] = dfTestAdmissions.apply(Admissions,colA=dColA,colB=dColB,axis=1) 
        #should we perform the SUM here? Ie SuM the total 1's i teh Row?
        
        if AdmissionHeading not in listOfColumns:
             listOfColumns.append(AdmissionHeading)

        j += 1

    #print(dfTestAdmissions[listOfColumns])
    
    #I am not convinced this is the way to go with this part. 
    
#     col_list = list(dfTestAdmissions)
#     #print(col_list)

#     #This section needs to be improved, as it needs to calculate at the end of each 'Section'
#     allHeadings = IdentifyColumns(df.copy(),  '')
#     print(allHeadings)
    
#     col_list = Retain__by_values(allHeadings, listOfColumns)
    
#     print (col_list)
    #col_list = delete__by_values( col_list, allHeadings )
    #print(col_list)

    #total =  dfTestAdmissions[col_list].sum(axis=1)

#     dfTestAdmissions['az_'+ str(AdmissionYears[i][-4:])] = dfTestAdmissions[col_list].sum(axis=1)

#     print('End of Section', i, dColA, dColB)
    
    
with pd.option_context('display.max_rows', 3, 'display.max_columns', None):  # more options can be specified also
    print(dfTestAdmissions)


In [ ]:
#loop all DischargeIndexes to calculate the required Numbers

listOfColumns = []

#reset the testing Data Frame.
dfTestDischarge = df.copy()

#numCols = AdmissionIndexes.shape[0]

#This is the first Year listing. ill need to loop over all Year Listing!

#print(AdmissionIndexes)
for i in range(len(DischargeIndexes.YearListing)  ): 
    if debug=='true':
        print(i, DischargeIndexes.YearListing[i])
        print('Start of New Section')

    DischargeYears = DischargeIndexes.YearListing[i]
  
    for j in range(len(DischargeYears) -1 ): 

        dColA = str(DischargeYears[j])
        dColB = str(DischargeYears[j + 1])
        
        if debug=='true':
            print('Identified Year Pairs:', DischargeYears[i], DischargeYears[j+1])       
            print('d', dColA, dColB)

        #This just means, select the first four characters of the String Variable
        DischargeHeading = 'D' +  str(dColA[-4:]) + '-' + str(dColB[-4:]) 
        
        if debug=='true':
            print(DischargeHeading)
        
        dfTestDischarge[DischargeHeading] = dfTestDischarge.apply(Discharges,colA=dColA,colB=dColB,axis=1) 

        if DischargeHeading not in listOfColumns:
             listOfColumns.append(DischargeHeading)

        j += 1
        
    #print(dfTestAdmissions[listOfColumns])

#     col_list = list(dfTestDischarge)
#     #print(col_list)

#     #This section needs to be improved, as it needs to calculate at the end of each 'Section'
#     allHeadings = IdentifyColumns(df.copy(),  '')

#     col_list = delete__by_values( col_list, allHeadings )
#     #print(col_list)

    #total =  dfTestAdmissions[col_list].sum(axis=1)

#     dfTestDischarge['dz_'+ str(DischargeYears[i][-4:])] = dfTestDischarge[col_list].sum(axis=1)

#     print('End of Section', i, dColA, dColB)
    

with pd.option_context('display.max_rows', 3, 'display.max_columns', None):  # more options can be specified also
    print(dfTestDischarge)


In [ ]:
#We now need to Count the Records per Row Groupoing.  I may be able to perform that as part of the previous iterations?

In [16]:
with pd.option_context('display.max_rows', 30, 'display.max_columns', None):  # more options can be specified also
    print(dfTestDischarge)

NameError: name 'dfTestDischarge' is not defined

In [18]:
#This is good  code, but I do not need to use it as of yet.

#Admission prefers leading '06'

FullList = df.head(0)
headings = FullList.columns

AdmissionPreference = ['06','07']
AdmissionMatching = [FullList for FullList in FullList if any(item in FullList for item in AdmissionPreference)]
AdmissionMatching.sort()

if debug=='true':
    print('Identified list of:', AdmissionMatching)

#these lines are identical to the ONE line of code above. Just not sure how to refactor as such.
indexes = []
for index, row in enumerate(headings):
    if row in AdmissionMatching:
        indexes.append(index)
print('Identified Indexes:' , indexes)      

Identified list of: ['201207', '201306', '201307', '201406']
Identified Indexes: [1, 12, 13, 24]
